In [2]:
import ray

In [3]:
# 라이브러리 임포트
import os
import time
import json
from datetime import datetime
import numpy as np
import pandas as pd
from fbprophet import Prophet
import pickle
import copy

In [4]:
import seaborn as sns
from plotly import express as px

In [5]:
from darts.models import *
from darts.models.forecasting import gradient_boosted_model
import darts.utils.timeseries_generation as tg
from darts.utils.likelihood_models import GaussianLikelihood
from darts.datasets import EnergyDataset
from darts.utils.missing_values import fill_missing_values
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.timeseries import TimeSeries
import warnings

warnings.filterwarnings("ignore")

/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [6]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: user 730 ms, sys: 165 ms, total: 895 ms
Wall time: 896 ms


%%time
''' 스무딩 '''
win_size = 3


for plc in plc_lst:
    df = df_dic[plc]
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')
    
    #print()
    
    # 컬럼별 스무딩
    for col in fct_lst:
        series = df[col]
        rolling = series.rolling(window=win_size)
        rolling_median = rolling.median()
        df_dic[plc][col] = rolling_median
    df_dic[plc] = df_dic[plc].dropna()
    df_dic[plc] = df_dic[plc].iloc[-1000:,:]

df = df_dic[plc_lst[0]]
sns.lineplot(x = df.ds , y = df.pH)

df.pH.iloc[2800:]

In [7]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

In [4]:
''' test  '''
i = plc_lst[0]
j = fct_lst[0]

plc_df = df_dic[i]
fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))


model = Prophet()
model.fit(fct_df)
forecast = model.predict(future)
#forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
#forecast = forecast[['ds', 'yhat']]


# TimeSeris 로 변환 하기 
dfts = TimeSeries.from_dataframe(fct_df,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True)

# 
my_model = gradient_boosted_model.LightGBMModel(
lags=60,
output_chunk_length=1,
random_state= 2022)

my_model.fit(dfts)

pred_series = my_model.predict( n = 699 )
forecast = pred_series.pd_dataframe()

forecast['ds'] = forecast.index
forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
forecast.columns = ['yhat', 'ds']
forecast = forecast.reset_index(drop=True)

for index, k in enumerate(list(forecast['ds'])):
    submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])
            

---
# 예측 시작

In [8]:
@ray.remote
def pred_all_columns(plc_df,i,fct_lst,submit):
    
    res_list = []
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        # 
        diff = datetime(2018,2,1) - fct_df.loc[0].ds
        if diff.days > 720:
            lag = 720
        else:
            lag = diff.days - 10

        my_model = gradient_boosted_model.LightGBMModel(
        lags=lag,
        output_chunk_length=1,
        random_state= 2022)
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = float(forecast[forecast['ds']==k]['yhat'])
            
        res_list.append( (i,j,forecast) )
    return res_list

In [9]:
fct_lst_ray = ray.put(fct_lst)
submit_ray = ray.put(submit)

In [10]:
task_list = [ pred_all_columns.remote(df_dic[x], x ,fct_lst_ray, submit_ray) for x in plc_lst ]

In [11]:
%%time
results = ray.get(task_list)

(pid=16515) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
(pid=16515)   warn(f"Failed to load image Python extension: {e}")
(pid=16521) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
(pid=16521)   warn(f"Failed to load image Python extension: {e}")
(pid=16508) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
(pid=16508)   warn(f"Failed to load image Python extension: {e}")
(pid=16525) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Fail

RayTaskError(ValueError): [36mray::pred_all_columns()[39m (pid=16496, ip=172.17.10.13)
  File "/tmp/ipykernel_47934/1232142960.py", line 33, in pred_all_columns
  File "/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/models/forecasting/gradient_boosted_model.py", line 162, in fit
    super().fit(
  File "/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/models/forecasting/regression_model.py", line 427, in fit
    self._fit_model(
  File "/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/models/forecasting/regression_model.py", line 321, in _fit_model
    training_samples, training_labels = self._create_lagged_data(
  File "/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/models/forecasting/regression_model.py", line 292, in _create_lagged_data
    raise_if(
  File "/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/logging.py", line 110, in raise_if
    raise_if_not(not condition, message, logger)
  File "/ssd/j

(pred_all_columns pid=16505) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/timeseries.py:3305: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pred_all_columns pid=16505)   if isinstance(time_idx, pd.Int64Index) and not isinstance(
(pred_all_columns pid=16518) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/timeseries.py:3305: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pred_all_columns pid=16518)   if isinstance(time_idx, pd.Int64Index) and not isinstance(
(pred_all_columns pid=16487) /ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/darts/timeseries.py:3305: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pred_all_columns pid=

In [49]:
for item in results:
    for x in item:
        i = x[0]
        j = x[1]
        forecast = x[2]
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])

/tmp/ipykernel_10600/3712719606.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])


In [50]:
# 추론 파일 저장
with open('./sub/baseline_median3.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)